### import packages

In [25]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### read in combined data

In [26]:
df_payments_combined = pd.read_pickle('../data/1_medicare_data/pickled_files/payments_combined.pkl')

In [27]:
df_payments_combined.head()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,hcpcs_code,hcpcs_description,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,23.0,23.0,23.0,72.680000,2015
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,18.0,18.0,18.0,135.850000,2015
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",59.0,58.0,59.0,101.365085,2015
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99222,"Initial hospital inpatient care, typically 50 ...",132.0,130.0,132.0,139.010455,2015
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",220.0,215.0,220.0,205.185955,2015


### Create new column for type of payment

In [28]:
conditions = [
    (df_payments_combined.place_of_service == 'O'),
    (df_payments_combined.entity_type_of_the_provider == 'I') & (df_payments_combined.place_of_service == 'F'),
    (df_payments_combined.entity_type_of_the_provider == 'O') & (df_payments_combined.place_of_service == 'F')]

choices = ['Doctor & Facility', 'Doctor Only', 'Facility Only']

In [29]:
%%time
df_payments_combined['payment_type'] = np.select(conditions, choices, default = 'unknown')

Wall time: 8.47 s


In [30]:
df_payments_combined.payment_type.value_counts()

Doctor & Facility    17733816
Doctor Only          11048040
Facility Only          278374
Name: payment_type, dtype: int64

# Pivot to get years side by side

### Payment Data

In [31]:
df_payments_combined.columns

Index(['national_provider_identifier',
       'last_name_organization_name_of_the_provider',
       'entity_type_of_the_provider', 'city_of_the_provider',
       'zip_code_of_the_provider', 'state_code_of_the_provider',
       'provider_type', 'place_of_service', 'hcpcs_code', 'hcpcs_description',
       'number_of_services', 'number_of_medicare_beneficiaries',
       'number_of_distinct_medicare_beneficiary_per_day_services',
       'average_medicare_allowed_amount', 'year', 'payment_type'],
      dtype='object')

In [32]:
pivot_index = ['national_provider_identifier',
               'entity_type_of_the_provider', 
               'place_of_service',
               'payment_type',
               'provider_type',
               'hcpcs_code',
               'hcpcs_description',
               'zip_code_of_the_provider', 
               'state_code_of_the_provider']

pivot_cols = ['year']

In [33]:
%%time
df_pmt_pvt = df_payments_combined.pivot_table(index = pivot_index, 
                                              columns = pivot_cols, 
                                              values = 'average_medicare_allowed_amount', 
                                              aggfunc=np.mean)
df_pmt_pvt = df_pmt_pvt.reset_index()

Wall time: 2min 42s


In [34]:
print(df_pmt_pvt.shape)
df_pmt_pvt.head()

(20793075, 12)


year,national_provider_identifier,entity_type_of_the_provider,place_of_service,payment_type,provider_type,hcpcs_code,hcpcs_description,zip_code_of_the_provider,state_code_of_the_provider,2015,2016,2017
0,1003000126,I,F,Doctor Only,Internal Medicine,99217,Hospital observation care discharge,215021854,MD,72.680000,72.743158,73.398800
1,1003000126,I,F,Doctor Only,Internal Medicine,99218,Hospital observation care typically 30 minutes,215021854,MD,NaN,NaN,100.080000
2,1003000126,I,F,Doctor Only,Internal Medicine,99219,Hospital observation care typically 50 minutes,215021854,MD,135.850000,135.010000,136.380000
3,1003000126,I,F,Doctor Only,Internal Medicine,99220,Hospital observation care typically 70 minutes...,215021854,MD,NaN,189.239565,190.363729
4,1003000126,I,F,Doctor Only,Internal Medicine,99221,"Initial hospital inpatient care, typically 30 ...",215021854,MD,101.365085,100.750000,101.680000


### Number of Services - not needed per new direction

In [35]:
'''
%%time
df_services_pvt = df_payments_combined.pivot_table(index = pivot_index, 
                                              columns = pivot_cols, 
                                              values = 'number_of_services', 
                                              aggfunc=np.mean)
df_services_pvt = df_services_pvt.reset_index()'''

"\n%%time\ndf_services_pvt = df_payments_combined.pivot_table(index = pivot_index, \n                                              columns = pivot_cols, \n                                              values = 'number_of_services', \n                                              aggfunc=np.mean)\ndf_services_pvt = df_services_pvt.reset_index()"

In [36]:
'''print(df_services_pvt.shape)
df_services_pvt.head()'''

'print(df_services_pvt.shape)\ndf_services_pvt.head()'

#### Number of beneficiaries - not needed per new direction

In [37]:
'''%%time
df_benefs_pvt = df_payments_combined.pivot_table(index = pivot_index, 
                                              columns = pivot_cols, 
                                              values = 'number_of_medicare_beneficiaries', 
                                              aggfunc=np.mean)
df_benefs_pvt = df_benefs_pvt.reset_index()'''

"%%time\ndf_benefs_pvt = df_payments_combined.pivot_table(index = pivot_index, \n                                              columns = pivot_cols, \n                                              values = 'number_of_medicare_beneficiaries', \n                                              aggfunc=np.mean)\ndf_benefs_pvt = df_benefs_pvt.reset_index()"

In [38]:
'''print(df_benefs_pvt.shape)
df_benefs_pvt.head()'''

'print(df_benefs_pvt.shape)\ndf_benefs_pvt.head()'

### Number of unique beneficieries per day

In [39]:
%%time
df_bpd_pvt = df_payments_combined.pivot_table(index = pivot_index, 
                                              columns = pivot_cols, 
                                              values = 'number_of_distinct_medicare_beneficiary_per_day_services', 
                                              aggfunc=np.mean)
df_bpd_pvt = df_bpd_pvt.reset_index()

Wall time: 2min 38s


In [40]:
print(df_bpd_pvt.shape)
df_bpd_pvt.head()

(20793075, 12)


year,national_provider_identifier,entity_type_of_the_provider,place_of_service,payment_type,provider_type,hcpcs_code,hcpcs_description,zip_code_of_the_provider,state_code_of_the_provider,2015,2016,2017
0,1003000126,I,F,Doctor Only,Internal Medicine,99217,Hospital observation care discharge,215021854,MD,23.0,57.0,100.0
1,1003000126,I,F,Doctor Only,Internal Medicine,99218,Hospital observation care typically 30 minutes,215021854,MD,NaN,NaN,26.0
2,1003000126,I,F,Doctor Only,Internal Medicine,99219,Hospital observation care typically 50 minutes,215021854,MD,18.0,38.0,52.0
3,1003000126,I,F,Doctor Only,Internal Medicine,99220,Hospital observation care typically 70 minutes...,215021854,MD,NaN,23.0,59.0
4,1003000126,I,F,Doctor Only,Internal Medicine,99221,"Initial hospital inpatient care, typically 30 ...",215021854,MD,59.0,20.0,16.0


# WILL NOT USE 
## EDA for # beneficiaries
Mary and Mahesh confirmed HCBB did suggest another metric is best for utilization. 
Assigned this metric to me in Kanban board.  Other team members to focus on other metrics.

In [41]:
#df_benefs_pvt.info(verbose = True)

In [42]:
#df_benefs_pvt.describe()

From descriptive stats:
- I means are always way higher than median, though numbers are in the same ballpark
- Looks like both metrics did lower for 2017

In [43]:
#data = df_benefs_pvt.iloc[:,9][df_benefs_pvt.iloc[:,9].notnull()]
#data.head()

In [44]:
#f, ax1 = plt.subplots()

#ax1.violinplot(data)

#ax2.violinplot(df_benefs_pvt['2016'], showmeans=True, showextrema=True, showmedians=True)

#ax3.violinplot(df_benefs_pvt['2016'], showmeans=True, showextrema=True, showmedians=True)

In [45]:
#data.hist()

Data is very much skewed to the right.  Will need to subset somehow to look at this. 

## EDA for payment

In [46]:
df_pmt_pvt['change_15_to_16'] = df_pmt_pvt[2016] - df_pmt_pvt[2015]
df_pmt_pvt['change_16_to_17'] = df_pmt_pvt[2017] - df_pmt_pvt[2016]
df_pmt_pvt['change_15_to_17'] = df_pmt_pvt[2017] - df_pmt_pvt[2015]

In [47]:
df_pmt_pvt.head()

year,national_provider_identifier,entity_type_of_the_provider,place_of_service,payment_type,provider_type,hcpcs_code,hcpcs_description,zip_code_of_the_provider,state_code_of_the_provider,2015,2016,2017,change_15_to_16,change_16_to_17,change_15_to_17
0,1003000126,I,F,Doctor Only,Internal Medicine,99217,Hospital observation care discharge,215021854,MD,72.680000,72.743158,73.398800,0.063158,0.655642,0.718800
1,1003000126,I,F,Doctor Only,Internal Medicine,99218,Hospital observation care typically 30 minutes,215021854,MD,NaN,NaN,100.080000,NaN,NaN,NaN
2,1003000126,I,F,Doctor Only,Internal Medicine,99219,Hospital observation care typically 50 minutes,215021854,MD,135.850000,135.010000,136.380000,-0.840000,1.370000,0.530000
3,1003000126,I,F,Doctor Only,Internal Medicine,99220,Hospital observation care typically 70 minutes...,215021854,MD,NaN,189.239565,190.363729,NaN,1.124164,NaN
4,1003000126,I,F,Doctor Only,Internal Medicine,99221,"Initial hospital inpatient care, typically 30 ...",215021854,MD,101.365085,100.750000,101.680000,-0.615085,0.930000,0.314915


In [48]:
df_pmt_pvt.describe()

year,national_provider_identifier,2015,2016,2017,change_15_to_16,change_16_to_17,change_15_to_17
count,2.079308e+07,9.497891e+06,9.714894e+06,9.847441e+06,3.026202e+06,3.883872e+06,2.913220e+06
mean,1.503401e+09,9.947302e+01,1.001832e+02,1.015993e+02,-8.862161e-01,5.967683e-01,-5.316401e-01
std,2.863523e+08,2.384662e+02,2.563474e+02,2.764955e+02,3.676731e+01,2.948526e+01,4.909106e+01
min,1.003000e+09,1.026528e-04,1.059603e-04,6.035380e-05,-2.445791e+04,-1.207591e+04,-3.035365e+04
25%,1.255400e+09,2.315282e+01,2.376625e+01,2.426084e+01,-7.950000e-01,-7.000000e-02,-5.469656e-01
50%,1.508809e+09,6.288879e+01,6.409979e+01,6.499766e+01,-8.000000e-02,2.100000e-01,8.000000e-02
75%,1.750349e+09,1.138798e+02,1.135600e+02,1.135044e+02,2.186936e-01,9.800000e-01,8.260000e-01
max,1.993000e+09,5.260147e+04,5.060338e+04,5.668435e+04,2.284054e+04,1.757079e+04,3.430538e+04


### Pivot, aggregating at HCPCS and payment type level

In [50]:
df_pmt_pvt.columns

Index(['national_provider_identifier',  'entity_type_of_the_provider',
                   'place_of_service',                 'payment_type',
                      'provider_type',                   'hcpcs_code',
                  'hcpcs_description',     'zip_code_of_the_provider',
         'state_code_of_the_provider',                           2015,
                                 2016,                           2017,
                    'change_15_to_16',              'change_16_to_17',
                    'change_15_to_17'],
      dtype='object', name='year')

In [54]:
df_pmt_pvt.columns = df_pmt_pvt.columns.astype(str)
df_pmt_pvt.columns

Index(['national_provider_identifier', 'entity_type_of_the_provider',
       'place_of_service', 'payment_type', 'provider_type', 'hcpcs_code',
       'hcpcs_description', 'zip_code_of_the_provider',
       'state_code_of_the_provider', '2015', '2016', '2017', 'change_15_to_16',
       'change_16_to_17', 'change_15_to_17'],
      dtype='object', name='year')

In [69]:
%%time
pivot_index2 = ['hcpcs_code',
                'hcpcs_description',
                'payment_type']
                

pivot_vals2 = ['2015',
               '2016',
               '2017',
               'change_15_to_16',
               'change_16_to_17',
               'change_15_to_17']

df_pmt_avg_pvt = df_pmt_pvt.pivot_table(index = pivot_index2,
                                        columns = None,
                                        values = pivot_vals2,
                                        aggfunc = [np.mean,np.median])

Wall time: 17.8 s


In [67]:
df_pmt_avg_pvt

mean  \
year                                                                                    2015   
hcpcs_code hcpcs_description                                  payment_type                     
0008M      Onc breast risk score                              Doctor & Facility          NaN   
00100      Anesthesia for procedure on salivary gland with... Doctor Only         266.582807   
00102      Anesthesia for procedure to repair lip defect p... Doctor Only          95.198939   
00103      Anesthesia for procedure on eyelid                 Doctor & Facility   255.076212   
                                                              Doctor Only         153.325203   
...                                                                                      ...   
R0075      Transportation of portable x-ray equipment and ... Facility Only        81.782458   
V2632      Posterior chamber intraocular lens                 Doctor & Facility   113.490000   
V2785      Processing, preserving and transporting corneal... Doctor & Facility          NaN   
                                                              Doctor Only        3735.000000   
                                                              Facility Only      3128.787733   

                                                                                              \
year                                                                                    2016   
hcpcs_code hcpcs_description                                  payment_type                     
0008M      Onc breast risk score                              Doctor & Facility  3419.419776   
00100      Anesthesia for procedure on salivary gland with... Doctor Only         290.975942   
00102      Anesthesia for procedure to repair lip defect p... Doctor Only                NaN   
00103      Anesthesia for procedure on eyelid                 Doctor & Facility   128.268843   
                                                              Doctor Only         148.615364   
...                                                                                      ...   
R0075      Transportation of portable x-ray equipment and ... Facility Only        80.305087   
V2632      Posterior chamber intraocular lens                 Doctor & Facility   113.040000   
V2785      Processing, preserving and transporting corneal... Doctor & Facility  1172.601191   
                                                              Doctor Only        3925.000000   
                                                              Facility Only      3784.231305   

                                                                                              \
year                                                                                    2017   
hcpcs_code hcpcs_description                                  payment_type                     
0008M      Onc breast risk score                              Doctor & Facility  3443.360000   
00100      Anesthesia for procedure on salivary gland with... Doctor Only         269.853470   
00102      Anesthesia for procedure to repair lip defect p... Doctor Only                NaN   
00103      Anesthesia for procedure on eyelid                 Doctor & Facility   118.002858   
                                                              Doctor Only         150.600096   
...                                                                                      ...   
R0075      Transportation of portable x-ray equipment and ... Facility Only        80.649999   
V2632      Posterior chamber intraocular lens                 Doctor & Facility   113.830000   
V2785      Processing, preserving and transporting corneal... Doctor & Facility  1198.226601   
                                                              Doctor Only                NaN   
                                                              Facility Only      3844.634734   

                                                     

In [73]:
df_pmt_pvt[df_pmt_pvt.hcpcs_code=='00103'].describe()

year,national_provider_identifier,2015,2016,2017,change_15_to_16,change_16_to_17,change_15_to_17
count,6.076000e+03,2441.000000,2567.000000,2463.000000,399.000000,842.000000,320.000000
mean,1.496940e+09,153.491939,148.108088,149.369263,-5.073272,1.715605,-3.986901
std,2.881421e+08,49.207127,47.956377,47.452792,14.021499,15.539966,17.280712
min,1.003024e+09,74.019412,59.754545,62.554211,-54.043743,-80.562174,-83.324048
25%,1.255322e+09,105.898824,101.580454,102.152000,-10.921750,-5.176057,-11.264643
50%,1.497745e+09,158.184107,153.505000,155.345385,-4.921071,0.692032,-2.699753
75%,1.740204e+09,189.027683,182.826879,183.254902,1.029494,6.765428,4.799938
max,1.992988e+09,382.622500,340.486471,333.142857,66.494943,90.961389,68.546287


averaging changes before grouping does not equal the change between averages. 